How to mock and pass through?



In [1]:
var execSync = require('child_process').execSync;
try {
    require.resolve('automock');
    require.resolve('bluebird');
} catch (e) {
    execSync('npm install automock bluebird');
}

var Promise = require('bluebird');
var automock = require('automock');
var promisifyMock = (req, dep) => {
    let ctx;
    ctx = automock.mockValue(req, {
        stubCreator: (name) => {
            var orig = Promise.promisify(req[name.split('.')[1]], {
                multiArgs: true,
                context: req
            });
            //console.log('create stub ' + name);
            return function () {
                console.log(name + ' (' + arguments[0] + ') in ' + JSON.stringify(dep));
                return orig.apply(null, arguments);
            };
        }
    });
    return ctx;
}
(promisifyMock);


[Function: promisifyMock]

Create an RPC server for every module?



In [2]:
var execSync = require('child_process').execSync;
try {
    require.resolve('socket.io');
} catch (e) {
    execSync('npm install socket.io');
}

var io = require('socket.io').listen(8099);
io.sockets.on('connection', function (socket) {
    socket.on('require', function (dep, fn) {
        socket.join(dep);
        var req = require(dep);
        let ctx;
        ctx = automock.mockValue(req, {
            stubCreator: (name) => {
                var orig = Promise.promisify(req[name.split('.')[1]], {
                    multiArgs: true,
                    context: req
                });
                return function () {
                    return orig.apply(null, arguments)
                        .then(function () {
                            socket.broadcast.to(dep).emit('resolve', arguments);
                        });
                };
            }
        });
        
        fn('done');
    });
    socket.on('call', function (name, arguments, resolve, reject) {
        console.log(name);
        ctx[name].apply(null, arguments)
            .then(function () {
                resolve.apply(null, arguments);
            })
            .catch(function () {
                reject.apply(null, arguments);
            });
    });
});

// TODO: output interactive angular component for controlling this server



Namespace {
  name: '/',
  server: 
   Server {
     nsps: { '/': [Circular] },
     _path: '/socket.io',
     _serveClient: true,
     parser: 
      { protocol: 4,
        types: [Object],
        CONNECT: 0,
        DISCONNECT: 1,
        EVENT: 2,
        ACK: 3,
        ERROR: 4,
        BINARY_EVENT: 5,
        BINARY_ACK: 6,
        Encoder: [Function: Encoder],
        Decoder: [Function: Decoder] },
     encoder: Encoder {},
     _adapter: [Function: Adapter],
     _origins: '*:*',
     sockets: [Circular],
     eio: 
      Server {
        clients: {},
        clientsCount: 0,
        wsEngine: 'uws',
        pingTimeout: 60000,
        pingInterval: 25000,
        upgradeTimeout: 10000,
        maxHttpBufferSize: 100000000,
        transports: [Object],
        allowUpgrades: true,
        allowRequest: [Function: bound ],
        cookie: 'io',
        cookiePath: '/',
        cookieHttpOnly: true,
        perMessageDeflate: [Object],
        httpCompression: [Object],
     

In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('socket.io-client');
} catch (e) {
    execSync('npm install socket.io-client');
}

var client = require('socket.io-client');
var sockifyClient = (dep, host) => {
    let ctx;
    ctx = automock.mockValue(require(dep), {
        stubCreator: (name) => {
            return function () {
                return new Promise((resolve, reject) => {
                    socket.emit('call', name.split('.')[1], arguments, function () {
                        resolve.apply(null, arguments);
                    }, function () {
                        reject.apply(null, arguments);
                    });
                });
            };
        }
    });
    //var fs = promisifyMock(ctx, dep);
    var socket = client.connect('http://localhost:8099');
    socket.on('connect', function () {
        socket.emit('require', dep, function () {
            
        });
        socket.on('resolve', function () {
            
        });
    });
    return ctx;
}
var fs = sockifyClient('fs');

$$.async();
fs.unlink('test.file')
    .catch(e => console.log(e))
    .then(() => fs.writeFile('test.file', 'some output'))
    .then(() => fs.readFile('test.file'))
    .then(r => r.toString())
    .then((e, r) => $$.sendResult(r))
    .catch(e => $$.sendError(e));


(node:36377) DeprecationWarning: sprintf() will be removed in the next major release, use the sprintf-js package instead.


unlink


ReferenceError: ctx is not defined
    at Socket.<anonymous> (evalmachine.<anonymous>:33:9)
    at emitMany (events.js:127:13)
    at Socket.emit (events.js:204:7)
    at /Users/briancullinan/jupytangular2/node_modules/socket.io/lib/socket.js:514:12
    at _combinedTickCallback (internal/process/next_tick.js:73:7)
    at process._tickCallback (internal/process/next_tick.js:104:9)